In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/../resort/rasotools-master/')
import rasotools
np.seterr(all='warn')
np.errstate(divide='raise')

## Calculating the estimate time for calculation:

In [2]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/*.nc')
files[6020]

'/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-54102_CEUAS_merged_v0.nc'

In [3]:
b = os.path.getsize(files[6020])
b

122974841

In [4]:
t = 2633.3687977790833/60
t

43.88947996298472

v2:

In [5]:
tnew = (1306.7448754310608 + 185.2546329498291) / 60
tnew

24.86665847301483

In [6]:
b/tnew

4945370.570535308

Speed: ~4.9 MB/(min thread)

In [7]:
dirsize = 0 
for i in files:
    dirsize += os.path.getsize(i)
dirsize

618842993516

In [8]:
mintime = (dirsize)/(b/tnew)

In [9]:
mintime/60/24 , 'days'

(86.89987387214894, 'days')

~ 4 days on 20 cores -> more cores available on the jet?

---
---

## v1:

In [10]:
filesout = glob.glob('/raid60/scratch/uli/converted/*')
filesout

['/raid60/scratch/uli/converted/0-20000-0-22152_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11019_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-54102_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11035_CEUAS_merged_v0.nc']

In [11]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[2]) as test:

    print(test)
    print(test.recordindices)
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    aa = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
aa

File: <HDF5 file "0-20000-0-54102_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/converted/0-20000-0-54102_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (22851,)
 - G | era5fb_______________________________________ : : 71
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type____________________________ : : 

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,9473.223633,1964-06-01 11:00:00,0,90000.000000,-2147483648,-2147483648,-131.210556,-170.020813,0.0,0.0
1,9473.223633,1964-07-15 11:00:00,0,88500.000000,-2147483648,-2147483648,-271.075256,-353.135101,0.0,0.0
2,9473.223633,1964-07-30 11:00:00,0,98500.000000,-2147483648,-2147483648,8763.082031,8730.548828,0.0,0.0
3,9473.223633,1964-08-04 23:00:00,0,89300.000000,-2147483648,-2147483648,-244.882065,-286.890900,0.0,0.0
4,9473.223633,1964-08-15 23:00:00,0,89900.000000,-2147483648,-2147483648,-172.731949,-172.108704,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4954544,89810.000000,2019-12-28 23:15:24,140,1.169487,-2147483648,-2147483648,0.506251,0.767653,NaN,NaN
4954545,90780.000000,2019-12-29 11:16:12,140,2.630227,-2147483648,-2147483648,3.134859,2.860141,NaN,NaN
4954546,91280.000000,2019-12-29 23:15:47,140,0.615662,-2147483648,-2147483648,1.913419,2.676003,NaN,NaN
4954547,91770.000000,2019-12-30 11:15:22,140,4.405243,-2147483648,-2147483648,3.490680,3.252057,NaN,NaN


In [12]:
b_aa = aa[aa['conversion_method'] > 0]
display(b_aa)
b_aa[np.isnan(b_aa['fg_depar@body'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.890000,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.690000,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.950000,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.000000,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.190000,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3149797,89810.0,2019-12-28 23:15:24,105,-3.825219,0,2,NaN,NaN,NaN,NaN
3149830,90780.0,2019-12-29 11:16:12,105,-5.392765,0,2,NaN,NaN,NaN,NaN
3149858,91280.0,2019-12-29 23:15:47,105,-0.788011,0,2,NaN,NaN,NaN,NaN
3149890,91770.0,2019-12-30 11:15:22,105,-5.440022,0,2,NaN,NaN,NaN,NaN


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1371,81700.0,1957-06-22 00:00:00,34,5.250000,0,3,NaN,4.47,NaN,NaN
1871,49900.0,1957-07-23 00:00:00,34,0.000000,0,3,NaN,-0.19,NaN,NaN
1872,50000.0,1957-07-23 00:00:00,34,0.000000,0,3,NaN,-0.17,NaN,NaN
1919,89200.0,1957-07-25 00:00:00,34,1.240000,0,3,NaN,1.02,NaN,NaN
1951,82700.0,1957-07-27 00:00:00,34,0.000000,0,3,NaN,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3149797,89810.0,2019-12-28 23:15:24,105,-3.825219,0,2,NaN,NaN,NaN,NaN
3149830,90780.0,2019-12-29 11:16:12,105,-5.392765,0,2,NaN,NaN,NaN,NaN
3149858,91280.0,2019-12-29 23:15:47,105,-0.788011,0,2,NaN,NaN,NaN,NaN
3149890,91770.0,2019-12-30 11:15:22,105,-5.440022,0,2,NaN,NaN,NaN,NaN


In [13]:
aa[np.isnan(aa['observation_value'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
5419,36100.0,1958-04-27 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32485,10000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32486,20000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32487,30000.0,1967-10-10 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
32493,10000.0,1967-10-11 00:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
322011,12800.0,2013-07-11 11:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322143,12500.0,2013-07-18 23:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322212,13900.0,2013-07-21 23:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN
322260,14000.0,2013-07-24 11:00:00,34,NaN,0,3,NaN,NaN,NaN,NaN


---
---

## v2:

In [14]:
filesout = glob.glob('/raid60/scratch/uli/converted_v2/*')
filesout

['/raid60/scratch/uli/converted_v2/0-20000-0-11019_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted_v2/0-20000-0-22152_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted_v2/0-20000-0-54102_CEUAS_merged_v0.nc']

In [15]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[-1]) as test:

    print(test)
    print(test.recordindices)
    print(test.recordindices['34'][:])
    print(test.recordindices['36'][:])
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    aaa = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
aaa

File: <HDF5 file "0-20000-0-54102_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/converted_v2/0-20000-0-54102_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (22851,)
 - G | era5fb_______________________________________ : : 71
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type____________________________ :

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,9473.223633,1964-06-01 11:00:00,0,90000.000000,-2147483648,-2147483648,-131.210556,-170.020813,0.0,0.0
1,9473.223633,1964-07-15 11:00:00,0,88500.000000,-2147483648,-2147483648,-271.075256,-353.135101,0.0,0.0
2,9473.223633,1964-07-30 11:00:00,0,98500.000000,-2147483648,-2147483648,8763.082031,8730.548828,0.0,0.0
3,9473.223633,1964-08-04 23:00:00,0,89300.000000,-2147483648,-2147483648,-244.882065,-286.890900,0.0,0.0
4,9473.223633,1964-08-15 23:00:00,0,89900.000000,-2147483648,-2147483648,-172.731949,-172.108704,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4954084,89810.000000,2019-12-28 23:15:24,140,1.169487,-2147483648,-2147483648,0.506251,0.767653,NaN,NaN
4954085,90780.000000,2019-12-29 11:16:12,140,2.630227,-2147483648,-2147483648,3.134859,2.860141,NaN,NaN
4954086,91280.000000,2019-12-29 23:15:47,140,0.615662,-2147483648,-2147483648,1.913419,2.676003,NaN,NaN
4954087,91770.000000,2019-12-30 11:15:22,140,4.405243,-2147483648,-2147483648,3.490680,3.252057,NaN,NaN


In [16]:
b_aaa = aaa[aaa['conversion_method'] > 0]
display(b_aaa)
b_aaa[np.isnan(b_aaa['fg_depar@body'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.890000,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.690000,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.950000,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.000000,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.190000,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3149337,89810.0,2019-12-28 23:15:24,105,-3.825219,0,2,NaN,NaN,NaN,NaN
3149370,90780.0,2019-12-29 11:16:12,105,-5.392765,0,2,NaN,NaN,NaN,NaN
3149398,91280.0,2019-12-29 23:15:47,105,-0.788011,0,2,NaN,NaN,NaN,NaN
3149430,91770.0,2019-12-30 11:15:22,105,-5.440022,0,2,NaN,NaN,NaN,NaN


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1371,81700.0,1957-06-22 00:00:00,34,5.250000,0,3,NaN,4.47,NaN,NaN
1871,49900.0,1957-07-23 00:00:00,34,0.000000,0,3,NaN,-0.19,NaN,NaN
1872,50000.0,1957-07-23 00:00:00,34,0.000000,0,3,NaN,-0.17,NaN,NaN
1919,89200.0,1957-07-25 00:00:00,34,1.240000,0,3,NaN,1.02,NaN,NaN
1951,82700.0,1957-07-27 00:00:00,34,0.000000,0,3,NaN,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3149337,89810.0,2019-12-28 23:15:24,105,-3.825219,0,2,NaN,NaN,NaN,NaN
3149370,90780.0,2019-12-29 11:16:12,105,-5.392765,0,2,NaN,NaN,NaN,NaN
3149398,91280.0,2019-12-29 23:15:47,105,-0.788011,0,2,NaN,NaN,NaN,NaN
3149430,91770.0,2019-12-30 11:15:22,105,-5.440022,0,2,NaN,NaN,NaN,NaN


In [17]:
aaa[np.isnan(aaa['observation_value'])]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body


---
---

In [18]:
converted_variables_v1 = 460584 
converted_variables_v2 = 460124 
nan_variables_v1 = 483
nan_variables_v2 = 0

print('gained variables: ', ((converted_variables_v2 - nan_variables_v2) - (converted_variables_v1 - nan_variables_v1)))

time_v1 = 1306.7448754310608 + 185.2546329498291
time_v2 = 1411.9842202663422 + 183.59471774101257
print('more time consumed: ', (time_v2 - time_v1)/60, 'min')

gained variables:  23
more time consumed:  1.7263238271077475 min


---
---

In [19]:
aaa[aaa['observed_variable'] == 34]

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
41,30000.0,1956-08-20 15:00:00,34,9.89,0,3,5.820000,3.810000,NaN,NaN
42,40000.0,1956-08-20 15:00:00,34,9.69,0,3,1.500000,-2.720000,NaN,NaN
43,50000.0,1956-08-20 15:00:00,34,6.95,0,3,-18.830002,-21.860001,NaN,NaN
44,70000.0,1956-08-20 15:00:00,34,1.00,0,3,-5.600000,-8.800000,NaN,NaN
45,85000.0,1956-08-20 15:00:00,34,3.19,0,3,-3.310000,-4.140000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
372761,85000.0,2019-12-30 23:15:09,34,8.59,0,3,-5.809999,-4.520000,NaN,NaN
372762,85500.0,2019-12-30 23:15:09,34,7.59,0,3,-6.000000,-4.690000,NaN,NaN
372763,88100.0,2019-12-30 23:15:09,34,6.79,0,3,-4.110000,-2.970000,NaN,NaN
372764,89100.0,2019-12-30 23:15:09,34,6.89,0,3,-3.880001,-2.830000,NaN,NaN


In [20]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-54102_CEUAS_merged_v0.nc') as test:
    ref = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
    arrayconverter = test.to_dataframe(groups='observations_table', variables=['observed_variable'])
    arrayconverter = arrayconverter.observed_variable.head(1).to_xarray()
ref

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,79190.484375,1956-08-20 03:00:00,106,150.000000,-2147483648,-2147483648,NaN,NaN,0.0,0.0
1,79190.484375,1956-08-20 03:00:00,107,7.000000,-2147483648,-2147483648,-0.236686,-0.458606,0.0,0.0
2,79190.484375,1956-08-20 03:00:00,104,-3.500000,-2147483648,-2147483648,-4.902144,-0.307024,0.0,0.0
3,79190.484375,1956-08-20 03:00:00,105,6.062178,-2147483648,-2147483648,-1.037373,-0.678424,0.0,0.0
4,84041.859375,1956-08-20 03:00:00,106,150.000000,-2147483648,-2147483648,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4498261,90840.000000,2019-12-31 11:15:17,117,9845.876953,-2147483648,-2147483648,72.100586,41.154060,NaN,NaN
4498262,90840.000000,2019-12-31 11:15:17,136,253.050003,-2147483648,-2147483648,NaN,NaN,NaN,NaN
4498263,90840.000000,2019-12-31 11:15:17,137,246.750000,-2147483648,-2147483648,NaN,NaN,NaN,NaN
4498264,90840.000000,2019-12-31 11:15:17,138,0.570813,-2147483648,-2147483648,0.296632,0.273383,NaN,NaN


In [21]:
test = ref[ref['z_coordinate'] == 30000.0][ref['date_time'] == '1956-08-20 15:00:00']
display(test)

<ipython-input-21-1dde28cc5541>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = ref[ref['z_coordinate'] == 30000.0][ref['date_time'] == '1956-08-20 15:00:00']


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
18,30000.0,1956-08-20 15:00:00,117,92868.976562,-2147483648,-2147483648,396.621185,436.673920,0.00000,0.00000
19,30000.0,1956-08-20 15:00:00,85,238.649994,-2147483648,-2147483648,2.755198,2.094388,1.35494,1.35494
20,30000.0,1956-08-20 15:00:00,38,0.360000,-2147483648,-2147483648,-0.295861,-0.172303,0.00000,0.00000


In [22]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd

array([9.89], dtype=float32)

In [23]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value'] - test[test['observed_variable'] == 85]['fg_depar@body']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']  - test[test['observed_variable'] == 38]['fg_depar@body']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd_corr = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd_corr

array([4.07], dtype=float32)

In [24]:
fg_dep = dpd - dpd_corr
fg_dep

array([5.82], dtype=float32)

In [25]:
arrayconverter.values = test[test['observed_variable'] == 85]['observation_value'] - test[test['observed_variable'] == 85]['an_depar@body']
t = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 38]['observation_value']  - test[test['observed_variable'] == 38]['an_depar@body']
rh = arrayconverter.copy()
arrayconverter.values = test[test['observed_variable'] == 85]['z_coordinate']
zc = arrayconverter.copy()
dpd_corr = rasotools.met.convert.to_dpd(temp=t,press=zc,rel_humi=rh).values
dpd_corr

array([6.08], dtype=float32)

In [26]:
an_dep = dpd - dpd_corr
an_dep

array([3.8100004], dtype=float32)

---
---

In [27]:
aaa[aaa['observed_variable'] == 85]['observation_value'].describe()

count    542125.000000
mean        233.411302
std          21.942441
min         173.500000
25%         217.300003
50%         223.300003
75%         247.899994
max         409.399994
Name: observation_value, dtype: float64

In [28]:
aaa[aaa['observed_variable'] == 85][aaa['observation_value'] > 273+60]

<ipython-input-28-7f9a8c9e6990>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aaa[aaa['observed_variable'] == 85][aaa['observation_value'] > 273+60]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1615248,100000.0,1977-04-12 12:00:00,85,358.549988,-2147483648,-2147483648,68.728920,65.781509,0.000000,0.000000
1617251,100000.0,1977-07-21 11:00:00,85,358.549988,-2147483648,-2147483648,53.794907,53.070599,0.000000,0.000000
1618206,70000.0,1977-09-03 23:00:00,85,344.149994,-2147483648,-2147483648,62.543777,62.842842,0.072587,0.072587
1629564,9590.0,1978-12-29 23:00:00,85,350.799988,-2147483648,-2147483648,131.994308,132.044220,0.405397,0.405397
1630266,23300.0,1979-01-25 23:00:00,85,349.399994,-2147483648,-2147483648,135.993622,135.850555,0.508642,0.508642
1640419,34500.0,1980-03-13 11:00:00,85,368.799988,-2147483648,-2147483648,144.914108,144.836792,0.442976,0.442976
1646227,3000.0,1980-11-05 11:00:00,85,358.200012,-2147483648,-2147483648,134.299271,133.178436,0.085514,0.085514
1651920,1000.0,1981-06-19 00:00:00,85,362.000000,-2147483648,-2147483648,124.410820,124.304565,0.280757,0.280757
1657254,1000.0,1982-01-23 12:00:00,85,361.600006,-2147483648,-2147483648,135.055359,135.592911,-0.465565,-0.465565
1664170,24500.0,1982-10-17 12:00:00,85,342.200012,-2147483648,-2147483648,128.494278,128.653015,0.631866,0.631866


In [29]:
ref[ref['observed_variable'] == 85][ref['observation_value'] > 273+50]

<ipython-input-29-33c289ee1a54>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref[ref['observed_variable'] == 85][ref['observation_value'] > 273+50]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
721408,100000.0,1977-04-12 12:00:00,85,358.549988,-2147483648,-2147483648,68.728920,65.781509,0.000000,0.000000
738471,100000.0,1977-07-21 11:00:00,85,358.549988,-2147483648,-2147483648,53.794907,53.070599,0.000000,0.000000
745415,70000.0,1977-09-03 23:00:00,85,344.149994,-2147483648,-2147483648,62.543777,62.842842,0.072587,0.072587
839748,9590.0,1978-12-29 23:00:00,85,350.799988,-2147483648,-2147483648,131.994308,132.044220,0.405397,0.405397
845087,23300.0,1979-01-25 23:00:00,85,349.399994,-2147483648,-2147483648,135.993622,135.850555,0.508642,0.508642
932071,34500.0,1980-03-13 11:00:00,85,368.799988,-2147483648,-2147483648,144.914108,144.836792,0.442976,0.442976
936984,3000.0,1980-04-07 11:00:00,85,324.399994,-2147483648,-2147483648,101.990768,101.790520,-0.370253,-0.370253
940868,25000.0,1980-04-26 00:00:00,85,332.000000,-2147483648,-2147483648,116.958542,115.394341,0.049384,0.049384
958041,15000.0,1980-07-12 00:00:00,85,327.000000,-2147483648,-2147483648,110.299744,110.325272,0.842118,0.842118
980576,3000.0,1980-11-05 11:00:00,85,358.200012,-2147483648,-2147483648,134.299271,133.178436,0.085514,0.085514


### corrected in the temperature, but no correction available for the dew point! Very wrong values in the data set:

In [30]:
ref[ref['observed_variable'] == 36][ref['observation_value'] > 400]

<ipython-input-30-4728a4da3272>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref[ref['observed_variable'] == 36][ref['observation_value'] > 400]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
1668809,2000.0,1989-01-07 11:00:00,36,409.399994,-2147483648,-2147483648,NaN,NaN,NaN,NaN


In [31]:
aaa[aaa['observed_variable'] == 36]['observation_value'].describe()

count    350979.000000
mean        228.019405
std          23.150486
min         135.300003
25%         209.899994
50%         219.699997
75%         246.199997
max         409.399994
Name: observation_value, dtype: float64

In [32]:
aaa[aaa['observed_variable'] == 36][aaa['observation_value'] > 400]

<ipython-input-32-e101584df9b4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aaa[aaa['observed_variable'] == 36][aaa['observation_value'] > 400]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
496254,2000.0,1989-01-07 11:00:00,36,409.399994,0,-2147483648,NaN,NaN,NaN,NaN


In [33]:
aaa[aaa['date_time'] == '1989-01-07 11:00:00'][aaa['z_coordinate'] == 2000.0]

<ipython-input-33-d954c8c00eb0>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aaa[aaa['date_time'] == '1989-01-07 11:00:00'][aaa['z_coordinate'] == 2000.0]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
144409,2000.0,1989-01-07 11:00:00,34,0.000000,0,2,NaN,NaN,NaN,NaN
496254,2000.0,1989-01-07 11:00:00,36,409.399994,0,-2147483648,NaN,NaN,NaN,NaN
870278,2000.0,1989-01-07 11:00:00,38,0.000000,0,2,NaN,NaN,NaN,NaN
1254751,2000.0,1989-01-07 11:00:00,39,0.000000,0,4,NaN,NaN,NaN,NaN
1724424,2000.0,1989-01-07 11:00:00,85,409.399994,-2147483648,-2147483648,184.154984,184.604919,-1.258051,-1.258051


---
---

In [34]:
aaa[aaa['observed_variable'] == 38]['observation_value'].describe()

count    374890.000000
mean          0.389537
std           0.224552
min           0.000000
25%           0.218266
50%           0.363151
75%           0.543582
max          14.268437
Name: observation_value, dtype: float64

In [35]:
aaa[aaa['observed_variable'] == 38][aaa['observation_value'] > 1]

<ipython-input-35-238b147a7803>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aaa[aaa['observed_variable'] == 38][aaa['observation_value'] > 1]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
766157,20000.0,1971-08-21 12:00:00,38,1.121865,-2147483648,-2147483648,0.905197,0.931630,NaN,NaN
766418,50000.0,1971-09-15 00:00:00,38,1.019705,-2147483648,-2147483648,0.865289,0.876684,NaN,NaN
769807,70000.0,1972-09-15 12:00:00,38,1.016086,-2147483648,-2147483648,0.811110,0.711070,NaN,NaN
770430,10000.0,1972-11-20 12:00:00,38,1.123224,-2147483648,-2147483648,1.111906,1.112919,NaN,NaN
801266,15000.0,1978-12-09 11:00:00,38,14.268437,-2147483648,-2147483648,14.240347,14.239548,NaN,NaN
803811,25000.0,1979-05-08 11:00:00,38,9.150748,-2147483648,-2147483648,8.808514,8.805318,NaN,NaN
803865,70000.0,1979-05-10 11:00:00,38,1.443589,-2147483648,-2147483648,0.790267,0.782120,NaN,NaN
804679,10000.0,1979-06-19 11:00:00,38,2.924753,-2147483648,-2147483648,2.907386,2.907102,NaN,NaN
804972,25000.0,1979-07-03 11:00:00,38,1.340657,-2147483648,-2147483648,1.275116,1.267282,NaN,NaN
806058,50000.0,1979-08-29 11:00:00,38,1.956470,-2147483648,-2147483648,1.529601,1.565617,NaN,NaN


In [36]:
ref[ref['observed_variable'] == 38][ref['observation_value'] > 1]

<ipython-input-36-66b371928ff7>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref[ref['observed_variable'] == 38][ref['observation_value'] > 1]


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
445441,20000.0,1971-08-21 12:00:00,38,1.121865,-2147483648,-2147483648,0.905197,0.931630,NaN,NaN
448244,50000.0,1971-09-15 00:00:00,38,1.019705,-2147483648,-2147483648,0.865289,0.876684,NaN,NaN
485764,70000.0,1972-09-15 12:00:00,38,1.016086,-2147483648,-2147483648,0.811110,0.711070,NaN,NaN
492689,10000.0,1972-11-20 12:00:00,38,1.123224,-2147483648,-2147483648,1.111906,1.112919,NaN,NaN
835595,15000.0,1978-12-09 11:00:00,38,14.268437,-2147483648,-2147483648,14.240347,14.239548,NaN,NaN
866673,25000.0,1979-05-08 11:00:00,38,9.150748,-2147483648,-2147483648,8.808514,8.805318,NaN,NaN
867230,70000.0,1979-05-10 11:00:00,38,1.443589,-2147483648,-2147483648,0.790267,0.782120,NaN,NaN
875670,10000.0,1979-06-19 11:00:00,38,2.924753,-2147483648,-2147483648,2.907386,2.907102,NaN,NaN
878823,25000.0,1979-07-03 11:00:00,38,1.340657,-2147483648,-2147483648,1.275116,1.267282,NaN,NaN
891116,50000.0,1979-08-29 11:00:00,38,1.956470,-2147483648,-2147483648,1.529601,1.565617,NaN,NaN
